## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-06-54-01 +0000,bbc,No imminent change to cash Isa tax-free allowance,https://www.bbc.com/news/articles/cqjq9yxkkrvo
1,2025-07-11-06-48-19 +0000,bbc,UK economy shrank unexpectedly in May,https://www.bbc.com/news/articles/cq6mvem8neno
2,2025-07-11-06-24-00 +0000,wsj,U.K. Economy Unexpectedly Slumps to Second Str...,https://www.wsj.com/economy/global/u-k-economy...
3,2025-07-11-06-22-59 +0000,nypost,Woman with ‘urgent message’ for President Trum...,https://nypost.com/2025/07/11/us-news/florida-...
4,2025-07-11-05-43-45 +0000,bbc,Justin Bieber stands on business on surprise n...,https://www.bbc.com/news/articles/ce8zwz0jp17o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2378/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,26
75,tariffs,7
45,rare,5
258,brazil,5
80,tariff,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,54
64,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...,53
77,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,51
45,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...,47
47,2025-07-10-18-53-57 +0000,nypost,Trump reflects on assassination attempt one ye...,https://nypost.com/2025/07/10/us-news/trump-re...,47


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,54,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
64,27,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...
20,22,2025-07-10-23-49-13 +0000,nypost,Hakeem Jeffries’ team blasts Zohran Mamdani’s ...,https://nypost.com/2025/07/10/us-news/hakeem-j...
45,21,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...
47,21,2025-07-10-18-53-57 +0000,nypost,Trump reflects on assassination attempt one ye...,https://nypost.com/2025/07/10/us-news/trump-re...
77,18,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
7,16,2025-07-11-05-25-20 +0000,nyt,Rare Earths Mining in Myanmar Is Polluting Tha...,https://www.nytimes.com/2025/07/11/world/asia/...
34,16,2025-07-10-21-00-31 +0000,nypost,"Texas AG Ken Paxton’s wife, state Sen. Angela,...",https://nypost.com/2025/07/10/us-news/texas-ag...
85,15,2025-07-10-10-50-45 +0000,bbc,Deadly new Russian drone and missile attack hi...,https://www.bbc.com/news/articles/cj3rvpe06rxo
35,14,2025-07-10-20-31-26 +0000,nypost,White House targets Fed chair Jerome Powell ov...,https://nypost.com/2025/07/10/us-news/white-ho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
